# Logging

Here, we set up a logger to log any outputs to a file.

In [1]:
import logging
import sys
import time

# create a logger and set level to debug
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

# set the logger to output to both stdout and a file
output_file_handler = logging.FileHandler("spici_run_%d.log"%time.time())
stdout_handler = logging.StreamHandler(sys.stdout)

# for the file, include the timestamp as well
formatter = logging.Formatter('%(asctime)s - %(message)s')
output_file_handler.setFormatter(formatter)

# add both stdout and file handlers to the logger
logger.addHandler(output_file_handler)
logger.addHandler(stdout_handler)

In [2]:
logger.debug("SPICI Run beginning of execution.")

SPICI Run beginning of execution.


# Status File Handling

In [3]:
spici_status_file = 'C:/Users/tonma/My Drive/Scripps/Jaffe Lab/Zooplankton ML/spici/spici_status.txt'
ml_status_file = 'C:/Users/tonma/My Drive/Scripps/Jaffe Lab/Zooplankton ML/ml_status.txt'
execution_file = 'C:/Users/tonma/My Drive/Scripps/Jaffe Lab/Zooplankton ML/spici/execution_status.txt'

In [4]:
def check_status(fname):
    f = open(fname, "r")
    text = f.read()
    f.close()
    return text == '1'

def update_status(fname,stat):
    f = open(fname, "w")
    f.write(stat)
    f.close()

def scipi_status(stat):
    update_status(spici_status_file,stat)
    
def execution_status(stat):
    update_status(execution_file,stat)

In [5]:
def check_ml_status():
    n = 0 # counter
    while(check_status(ml_status_file)):
        time.sleep(1)
        if n%60==0: # log to file every minute
            logger.debug("Waiting for ML script: %d mins."%(n/60))
        n = n+1
    

# Helper Functions

In [6]:
def generateDatefile(date_start, date_end):
    fname = 'C:/Users/tonma/My Drive/Scripps/Jaffe Lab/Zooplankton ML/spici/spici_date.txt'
    format_str = '%s,%s,0.5,5.0,SPC2'%(date_start,date_end)
    f = open(fname, "w")
    f.write(format_str)
    f.close()    

# Implementation

## Setup

In [7]:
import pandas as pd
import numpy as np
import time
import shutil
import os
import subprocess
import sys
from io import StringIO

In [8]:
import errno, os, stat, shutil

def handleRemoveReadonly(func, path, exc):
    excvalue = exc[1]
    if func in (os.rmdir, os.remove) and excvalue.errno == errno.EACCES:
        os.chmod(path, stat.S_IRWXU| stat.S_IRWXG| stat.S_IRWXO) # 0777
        func(path)
    else:
        raise

In [9]:
def execute(cmd):
    with subprocess.Popen(cmd, stdout=subprocess.PIPE, bufsize=1,
                          universal_newlines=True) as p, StringIO() as buf:
        for line in p.stdout:
            logger.debug(line)
            buf.write(line)
        output = buf.getvalue()
    rc = p.returncode
    return rc
        
def run_spici(date_file,meta_file,image_folder,download=False):
    command = ('python spc_go.py --search-param-file=%s '%date_file +
    '--image-output-path=%s '%image_folder +
    '--meta-output-path=%s ' %meta_file
    )
    if download:
        command = command + '-d'
    logger.debug(command)
    execute(command)


In [10]:
# enter start date and number of days here
date_start = '2018-06-30'
h_interval = 3 # interval between each data point in hours
n_day = 1
n_interval = 24/h_interval*n_day+1

date_fmt = "%Y-%m-%d %H:%M:%S" # for formatting date string

start_dt = pd.to_datetime(date_start)
# delta_dt = pd.to_timedelta(np.arange(n_day), unit="D")
delta_dt = pd.to_timedelta(np.linspace(0,n_day*24,num=int(n_interval)),unit='h')
date_list = start_dt+delta_dt

# for logging
logger.debug("Start date: %s."%start_dt.strftime(date_fmt))
logger.debug("Number of days = %d."%n_day)

Start date: 2018-06-30 00:00:00.
Number of days = 1.


In [11]:
date_file = 'spici_date.txt'
meta_file = 'scipi_meta.csv'
image_folder = 'images'

logger.debug("Date file: %s."%date_file)
logger.debug("Meta file: %s."%meta_file)
logger.debug("Image Folder: %s."%image_folder)

Date file: spici_date.txt.
Meta file: scipi_meta.csv.
Image Folder: images.


## Execution

In [12]:
execution_status("1") # update execution status to 1
# iterate through all dates on the list
for i in range(len(date_list)-1): # note that the last one is skipped
    cur_date = date_list[i]
    next_date = date_list[i+1]
    cur_date_str = cur_date.strftime(date_fmt)
    next_date_str = next_date.strftime(date_fmt)
    
    # first check ML flag
    check_ml_status()
    
    # start running scipi when ML script is inactive
    scipi_status("1") # set flag to 1
    
    # start by deleting old images if the folder exists
    if os.path.exists(image_folder):
        logger.debug("Found old image folder. Perform deletion.")
        shutil.rmtree(image_folder,ignore_errors=False, onerror=handleRemoveReadonly)
        logger.debug("Finish deleting old image folder.")
    
    
    logger.debug("Start downloading images from %s."%cur_date_str)
    # generate date file
    generateDatefile(cur_date_str,next_date_str)
    # run scipi
    run_spici(date_file,meta_file,image_folder,download=True)
    scipi_status("0") # set flag back to 0 after done
    time.sleep(15) # wait a minute for ML script to register the new flag

execution_status("0") # update execution status to 0    

Found old image folder. Perform deletion.
Finish deleting old image folder.
Start downloading images from 2018-06-30 00:00:00.
python spc_go.py --search-param-file=spici_date.txt --image-output-path=images --meta-output-path=scipi_meta.csv -d

Number of urls: 1

Number of results: 5446

Number of images: 500

Number of images: 446

Number of images: 500

Number of images: 500

Number of images: 500

Number of images: 500

Number of images: 500

Number of images: 500

Number of images: 500

Number of images: 500

Number of images: 500

Downloaded 5446 images in 0:00:06.138766 (887 images/sec)

Waiting for ML script: 0 mins.
Found old image folder. Perform deletion.
Finish deleting old image folder.
Start downloading images from 2018-06-30 03:00:00.
python spc_go.py --search-param-file=spici_date.txt --image-output-path=images --meta-output-path=scipi_meta.csv -d

Number of urls: 1

Number of results: 2334

Number of images: 334

Number of images: 500

Number of images: 500

Number of im